<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [ ]:
# Descargar la carpeta de dataset
import os
import platform
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip
else:
    print("El dataset ya se encuentra descargado")

--2024-07-09 12:31:02--  http://songs_dataset.zip/
Resolving songs_dataset.zip (songs_dataset.zip)... failed: Name or service not known.
wget: unable to resolve host address ‘songs_dataset.zip’
--2024-07-09 12:31:02--  https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip [following]
--2024-07-09 12:31:03--  https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.

In [ ]:
# Posibles bandas
os.listdir("./songs_dataset/")

['Kanye_West.txt',
 'jimi-hendrix.txt',
 'al-green.txt',
 'eminem.txt',
 'leonard-cohen.txt',
 'nicki-minaj.txt',
 'prince.txt',
 'r-kelly.txt',
 'beatles.txt',
 'paul-simon.txt',
 'lady-gaga.txt',
 'lil-wayne.txt',
 'bjork.txt',
 'adele.txt',
 'patti-smith.txt',
 'nirvana.txt',
 'cake.txt',
 'radiohead.txt',
 'alicia-keys.txt',
 'notorious-big.txt',
 'johnny-cash.txt',
 'amy-winehouse.txt',
 'bieber.txt',
 'bruno-mars.txt',
 'dolly-parton.txt',
 'nickelback.txt',
 'bruce-springsteen.txt',
 'nursery_rhymes.txt',
 'disney.txt',
 'rihanna.txt',
 'dj-khaled.txt',
 'notorious_big.txt',
 'kanye-west.txt',
 'dr-seuss.txt',
 'michael-jackson.txt',
 'drake.txt',
 'bob-marley.txt',
 'britney-spears.txt',
 'lin-manuel-miranda.txt',
 'lorde.txt',
 'ludacris.txt',
 'bob-dylan.txt',
 'blink-182.txt',
 'kanye.txt',
 'Lil_Wayne.txt',
 'joni-mitchell.txt',
 'dickinson.txt',
 'janisjoplin.txt',
 'missy-elliott.txt']

In [ ]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('songs_dataset/jimi-hendrix.txt', sep='/n', header=None)
df.head()

<ipython-input-42-5042aa551c22>:2: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



,0
0,There must be some kind of way outta here
1,Said the joker to the thief
2,There's too much confusion
3,"I can't get no relief Business men, they drink..."
4,Plowman dig my earth


In [ ]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 2033


### 1 - Preprocesamiento

In [ ]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [ ]:
# Demos un vistazo
sentence_tokens[:2]

[['there', 'must', 'be', 'some', 'kind', 'of', 'way', 'outta', 'here'],
 ['said', 'the', 'joker', 'to', 'the', 'thief']]

### 2 - Crear los vectores (word2vec)

In [ ]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [ ]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [ ]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [ ]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 2033


In [ ]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 483


### 3 - Entrenar embeddings

In [ ]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 133691.125
Loss after epoch 1: 65327.5
Loss after epoch 2: 64197.25
Loss after epoch 3: 64902.46875
Loss after epoch 4: 64708.1875
Loss after epoch 5: 63767.53125
Loss after epoch 6: 64318.0
Loss after epoch 7: 65705.75
Loss after epoch 8: 64429.0
Loss after epoch 9: 64743.5625
Loss after epoch 10: 64815.125
Loss after epoch 11: 65451.3125
Loss after epoch 12: 64556.1875
Loss after epoch 13: 64384.4375
Loss after epoch 14: 63444.0625
Loss after epoch 15: 56788.25
Loss after epoch 16: 55191.375
Loss after epoch 17: 55348.375
Loss after epoch 18: 55251.375
Loss after epoch 19: 55418.875


(156786, 285540)

### 4 - Ensayar

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["darling"], topn=10)

[('thought', 0.9930726289749146),
 ('scene', 0.9928821921348572),
 ('full', 0.9928471446037292),
 ('feeling', 0.9925820231437683),
 ('feet', 0.992576003074646),
 ('man', 0.992499053478241),
 ('turned', 0.9923893213272095),
 ('body', 0.9923678636550903),
 ('fall', 0.9923567175865173),
 ('fight', 0.9921563267707825)]

In [ ]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["love"], topn=10)

[("floatin'", -0.26228073239326477),
 ("you've", -0.2762462794780731),
 ('got', -0.29888230562210083),
 ("'round", -0.30804571509361267),
 ('go', -0.39515531063079834),
 ('stop', -0.40961435437202454),
 ('mine', -0.41921401023864746),
 ("nat'rally", -0.42443859577178955),
 ('johnny', -0.44523075222969055),
 ('voodoo', -0.4607124626636505)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["lady"], topn=10)

[('ooh', 0.9784362316131592),
 ('foxy', 0.9703792929649353),
 ('ow', 0.9631498456001282),
 ('yeah', 0.9588172435760498),
 ("you're", 0.9576972126960754),
 ('two', 0.9561972618103027),
 ('wrong', 0.9561073184013367),
 ('caught', 0.9557348489761353),
 ('hmmm', 0.9554049968719482),
 ('goes', 0.9553000330924988)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["pain"], topn=5)

[('fool', 0.9974985122680664),
 ('ooo', 0.9969948530197144),
 ('mess', 0.9967654347419739),
 ('use', 0.9965922236442566),
 ('waterfall', 0.9962754845619202)]

In [ ]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["all"])

[('voodoo', -0.4738563895225525),
 ('stop', -0.4934901297092438),
 ('go', -0.49757564067840576),
 ('johnny', -0.5330703854560852),
 ('lord', -0.5359761118888855),
 ('train', -0.5541788935661316),
 ('hear', -0.571855366230011),
 ("i'm", -0.6065425276756287),
 ('what', -0.6161960959434509),
 ('know', -0.6215039491653442)]

In [ ]:
# el método `get_vector` permite obtener los vectores:
vector_love = w2v_model.wv.get_vector("love")
print(vector_love)

[ 0.01049728  0.19773228  0.0458908   0.10465935  0.19373137 -0.196871
 -0.03119683  0.30169213 -0.05386839  0.06960212  0.01529263  0.01674076
  0.05928168  0.2508703   0.12619588 -0.11618598  0.21198352 -0.17084923
 -0.09990938  0.02224028 -0.09447619 -0.10073262  0.19699873  0.19398656
  0.21742348 -0.04301547 -0.12328677  0.12229904  0.00153714 -0.09613281
 -0.0788701   0.00517382  0.07982669 -0.02587628  0.16242616  0.18978362
 -0.01234346 -0.24535781 -0.21922435  0.08151508 -0.18058376 -0.00439726
  0.15874973 -0.05946439 -0.01341624 -0.0441635  -0.03223175  0.16884486
 -0.21075454 -0.03336628  0.06644834 -0.09050999 -0.34086668 -0.1463878
  0.11465127  0.03434211 -0.07963278 -0.00336497 -0.08911004 -0.16748603
 -0.04264742 -0.04899375  0.14596641  0.08518437  0.06701578  0.07878257
  0.2697136   0.019994    0.06646239  0.20964155  0.1449899   0.09719054
  0.26551503  0.05487012  0.01957942  0.11293309  0.06735156 -0.2885084
 -0.1301144   0.12629697 -0.05507597 -0.1730314  -0.090

In [ ]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_love)

[('love', 1.0),
 ('make', 0.9668053984642029),
 ('thing', 0.8959028124809265),
 ('wild', 0.8948818445205688),
 ('everything', 0.890697181224823),
 ('groovy', 0.8709408640861511),
 ('will', 0.8411408066749573),
 ('heart', 0.8352189064025879),
 ("you'll", 0.8206409811973572),
 ('bold', 0.8081518411636353)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["love"], topn=10)

[('make', 0.9668053984642029),
 ('thing', 0.8959028124809265),
 ('wild', 0.8948818445205688),
 ('everything', 0.890697181224823),
 ('groovy', 0.8709408640861511),
 ('will', 0.8411407470703125),
 ('heart', 0.8352188467979431),
 ("you'll", 0.8206409811973572),
 ('bold', 0.8081518411636353),
 ('as', 0.8069232106208801)]

### 5 - Visualizar agrupación de vectores

In [ ]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [ ]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [ ]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

In [ ]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

Importar y cargar el dataset elegido:

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving archive.zip to archive.zip


In [ ]:
import zipfile
import os

# Ruta del archivo zip subido (ajusta el nombre según el archivo que hayas subido)
zip_path = 'archive.zip'

# Crear una carpeta para extraer los archivos
extract_folder = 'extracted_files'
os.makedirs(extract_folder, exist_ok=True)

# Descomprimir el archivo
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

# Listar los archivos descomprimidos
print("Archivos descomprimidos:")
os.listdir(extract_folder)


Archivos descomprimidos:


['Beginner_Reviews_dataset.csv']

In [ ]:

# Ruta del archivo CSV dentro de la carpeta descomprimida (ajusta el nombre según sea necesario)
csv_file_path = os.path.join(extract_folder, 'Beginner_Reviews_dataset.csv')
data = pd.read_csv(csv_file_path)

# Ver las primeras filas del dataset
df = data.drop('Unnamed: 0', axis=1)
df = df.drop('label', axis=1)

df.tail()


,sentence
995,I think food should have flavor and texture an...
996,Appetite instantly gone.
997,Overall I was not impressed and would not go b...
998,"The whole experience was underwhelming, and I ..."
999,"Then, as if I hadn't wasted enough of my life ..."


In [ ]:
print('Cantidad de documentos: ', df.shape[0])

Cantidad de documentos:  1000


In [ ]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [ ]:
sentence_tokens[:2]

[['wow', 'loved', 'this', 'place'], ['crust', 'is', 'not', 'good']]

In [ ]:
from collections import Counter

# Aplanar el corpus a una lista de palabras
all_words = [word for document in sentence_tokens for word in document]

# Contar la frecuencia de las palabras
word_counts = Counter(all_words)

# Obtener las palabras más comunes
most_common_words = word_counts.most_common(10)

print("Palabras más comunes:", most_common_words)

Palabras más comunes: [('the', 585), ('and', 393), ('i', 305), ('was', 295), ('a', 237), ('to', 220), ('is', 171), ('this', 143), ('it', 134), ('of', 127)]


In [ ]:
from gensim.models.callbacks import CallbackAny2Vec
# Sobrecargamos el callback para imprimir el loss de cada época
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [ ]:
# Crear el modelo generador de vectores
w2v_model = Word2Vec(min_count=5, window=2, vector_size = 200, negative=10, workers=1, sg=1) # sg1: skipgram

In [ ]:
w2v_model.build_vocab(sentence_tokens)

In [ ]:
# imprimir cantidad de palabras encontradas en el corpus
print('Cantidad de palabras distintas en el corpus: ', len(w2v_model.wv.index_to_key))


Cantidad de palabras distintas en el corpus:  349


In [ ]:
# Cantidad de filas o docs en el corpus
print('Cantidad de docs: ', w2v_model.corpus_count)

Cantidad de docs:  1000


In [ ]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 79419.1640625
Loss after epoch 1: 43486.90625
Loss after epoch 2: 37347.2421875
Loss after epoch 3: 36405.765625
Loss after epoch 4: 37400.078125
Loss after epoch 5: 36800.875
Loss after epoch 6: 36586.46875
Loss after epoch 7: 36917.25
Loss after epoch 8: 35845.71875
Loss after epoch 9: 36793.40625
Loss after epoch 10: 37350.375
Loss after epoch 11: 37259.6875
Loss after epoch 12: 37637.25
Loss after epoch 13: 36907.875
Loss after epoch 14: 36920.0
Loss after epoch 15: 37402.75
Loss after epoch 16: 37109.875
Loss after epoch 17: 37612.375
Loss after epoch 18: 36977.9375
Loss after epoch 19: 36631.0625


(98323, 218120)

In [ ]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [ ]:
# Palabras que MÁS se relacionan con "bad"
similar_to_bad = w2v_model.wv.most_similar(positive=["bad"], topn=10)
print("Palabras más relacionadas con 'bad':", similar_to_bad)


Palabras más relacionadas con 'bad': [('waitress', 0.9978378415107727), ('terrible', 0.9978376626968384), ('tender', 0.9977381229400635), ('slow', 0.9977093935012817), ('tasty', 0.9976891279220581), ('ambiance', 0.9976723194122314), ('wonderful', 0.9974862933158875), ('prices', 0.9974544048309326), ('excellent', 0.9974522590637207), ('waiter', 0.9974120855331421)]


In [ ]:
# Palabras que MÁS se relacionan con "amazing"
similar_to_amazing = w2v_model.wv.most_similar(positive=["amazing"], topn=10)
print("Palabras más relacionadas con 'amazing':", similar_to_amazing)


Palabras más relacionadas con 'amazing': [('burger', 0.9977477192878723), ('wonderful', 0.9976289868354797), ('perfect', 0.9976158738136292), ('terrible', 0.9976111650466919), ('waitress', 0.9975799322128296), ('bland', 0.9975767135620117), ('steak', 0.9975324273109436), ('warm', 0.997504711151123), ('sauce', 0.9974865913391113), ('tasty', 0.9974698424339294)]


In [ ]:

# Palabras que MENOS se relacionan con "worst"
similar_to_worst = w2v_model.wv.most_similar(negative=["worst"], topn=10)
print("Palabras menos relacionadas con 'worst':", similar_to_worst)


Palabras menos relacionadas con 'worst': [('back', -0.7760929465293884), ('be', -0.8094397187232971), ('will', -0.8104535341262817), ('go', -0.834240198135376), ("won't", -0.8448553681373596), ('again', -0.8887776136398315), ('never', -0.9016766548156738), ('going', -0.9049822092056274), ('would', -0.934967577457428), ('you', -0.9349810481071472)]


In [ ]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [ ]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab